# Traveling Salesman Problem
### Definition
<p>
    Given a collection of cities, the traveling salesman must determine the
    shortest route that will enable him to visit each city precisely once and
    then return back to his starting point.
    The distance between each city is given and is assumed to be the same in
    both directions.
    Objective is to minimize the total distance to be travelled.
<p/>

<p>
    This type of problem frequently occurs when coding the AI for strategy
    games. Often it’s necessary to create the shortest path for a unit that
    will start at one waypoint, end at another, and pass through several
    predefined areas along the way, to pick up resources, food, energy, and so
    on. It can also be used as part of the route planning AI for a Quake-like
    FPS bot.
    Some of the landmarks in TSP solving, starting from the 1950s.
</p>

In [229]:
import numpy as np
from copy import deepcopy

In [230]:
def permutation_crossover(parent1, parent2, crossover_rate):
    
    if np.random.rand() < crossover_rate:
        return deepcopy(parent1), deepcopy(parent2)
    
    chromosone_length = len(parent1)
    # random number between [0, chromosone_length)
    cp1 = np.random.randint(0, chromosone_length) 
    cp2 = np.random.randint(0, chromosone_length)

    while cp1 == cp2:
        cp2 = np.random.randint(0, chromosone_length)
    
    child1, child2 = deepcopy(parent1), deepcopy(parent2)
    value_to_map1 = child1[cp1:cp2]
    value_to_map2 = child2[cp1:cp2]
    
    for value1, value2 in zip(value_to_map1, value_to_map2):
        for i, city in enumerate(child1):       
            if city == value1:
                child1[i] = value2
            elif city == value2:
                child1[i] = value1
        
        for i, city in enumerate(child2):
            if city == value1:
                child2[i] = value2
            elif city == value2:
                child2[i] = value1
                
    return child1, child2

In [231]:
def insertion_mutation(chromosone, mutation_rate):
    
    if np.random.rand() < mutation_rate:
        return chromosone
    
    chromosone_length = len(chromosone)
    selected_index = np.random.randint(0, chromosone_length)
    insertion_index = np.random.randint(0, chromosone_length)

    while insertion_index == selected_index :
        insertion_index = np.random.randint(0, chromosone_length)
    
    gene = chromosone[selected_index]
    step = -1 if insertion_index < selected_index else 1
    
    for i in range(selected_index, insertion_index, step):
            chromosone[i] = chromosone[i+step]
    
    chromosone[insertion_index] = gene      

    return chromosone

In [232]:
def find_cost(cost_matrix, individual):
    cost, stop = 0, len(individual)-1
    for i in range(stop):
        x, y = individual[i], individual[i+1]
        cost += cost_matrix[x, y]
    return cost

In [233]:
def compute_fitness_score(cost_matrix, population):
    fitness = np.zeros(population.shape[0])
    for i, individual in enumerate(population):
        fitness[i] = find_cost(cost_matrix, individual)
    fitness = fitness.max() - fitness
    return fitness

In [234]:
def sigma_scaling(fitness_score):
    average = fitness_score.mean()
    std_2 = 2 * fitness_score.std()
    new_fitness = (fitness_score - average) / std_2
    return new_fitness

In [235]:
def tournament_selection(population, fitness_score, k):
    tournament = np.empty(k, dtype=int)
    pop_size = population.shape[0] 
    for i in range(k):
        random_no = np.random.randint(0, pop_size)
        while random_no in tournament:
            random_no = np.random.randint(0, pop_size)
        tournament[i] = random_no

    best = max(tournament, key=lambda x: fitness_score[x])

    return best

In [236]:
def mating(population, fitness_score, crossover_rate, mutation_rate):
    pop_size = population.shape[0]
    new_population = np.empty(population.shape, dtype=int)
    k = 2 # number of individual for tournament selection
    for i in range(0, pop_size, 2): # In each iteration we select two parents
        idx_1 = tournament_selection(population, fitness_score, k)
        idx_2 = tournament_selection(population, fitness_score, k)
        while idx_2 == idx_1:
            idx_2 = tournament_selection(population, fitness_score, k)
        
        parent1, parent2 = population[idx_1], population[idx_2]
        child1, child2 = permutation_crossover(parent1, parent2, crossover_rate)
        child1 = insertion_mutation(child1, mutation_rate)
        child2 = insertion_mutation(child2, mutation_rate)
        new_population[i] = child1
        new_population[i+1] = child2
        
    return new_population

In [237]:
# Random permutation for generating initial population
def initialize_population(pop_size, number_cities):
    population = np.full((pop_size, number_cities), -1)
    city = np.array([i for i in range(number_cities)])
    for i in range(pop_size):
       population[i] =  np.random.permutation(city)
    return population

In [238]:
# Parameters
total_cities = 6
population_size = 100
# population_size = np.math.factorial(total_cities)
epochs = 10_000
show_info_after = 1_000
crossover_rate = 0.7
mutation_rate = 0.001

In [239]:
population = initialize_population(population_size, total_cities)

In [240]:
# Initial Population
population

array([[2, 5, 3, 4, 0, 1],
       [5, 3, 2, 4, 0, 1],
       [3, 4, 1, 0, 5, 2],
       [2, 5, 1, 0, 4, 3],
       [1, 4, 0, 2, 3, 5],
       [0, 5, 4, 2, 3, 1],
       [5, 3, 1, 4, 2, 0],
       [0, 1, 2, 3, 4, 5],
       [1, 5, 3, 2, 0, 4],
       [2, 5, 0, 1, 3, 4],
       [5, 0, 1, 2, 4, 3],
       [2, 0, 3, 1, 4, 5],
       [0, 1, 4, 5, 2, 3],
       [5, 3, 4, 2, 0, 1],
       [4, 2, 5, 1, 0, 3],
       [0, 2, 3, 4, 1, 5],
       [3, 2, 1, 5, 4, 0],
       [4, 0, 2, 1, 3, 5],
       [3, 2, 5, 1, 0, 4],
       [2, 3, 0, 4, 5, 1],
       [5, 2, 4, 1, 0, 3],
       [4, 5, 3, 0, 1, 2],
       [2, 4, 0, 3, 5, 1],
       [2, 4, 0, 1, 5, 3],
       [1, 3, 5, 2, 0, 4],
       [5, 1, 0, 4, 3, 2],
       [1, 2, 0, 5, 4, 3],
       [1, 0, 3, 5, 2, 4],
       [4, 2, 1, 3, 0, 5],
       [4, 2, 1, 5, 0, 3],
       [3, 0, 5, 2, 1, 4],
       [2, 0, 5, 4, 1, 3],
       [1, 4, 2, 0, 3, 5],
       [5, 3, 2, 0, 1, 4],
       [0, 4, 3, 2, 1, 5],
       [5, 0, 1, 4, 2, 3],
       [4, 2, 1, 0, 3, 5],
 

In [241]:
# This costs matrix show the distance between each city 
# City are 0, 1, 2, 3, 4, 5
costs = np.array([
    [  0, 105,  95, 340, 256, 292],
    [105,   0, 372, 288, 145, 309],
    [ 95, 372,   0, 155, 200, 188],
    [340, 288, 155,   0, 244, 327],
    [256, 145, 200, 244,   0,  98],
    [292, 309, 188, 327,  98,   0]
])

In [242]:
# Example: Distance between city 2 and city 5
print(f'Distance between city 2 and city 5 is : {costs[2, 5]}')
# Example: Distance between city 0 and city 4
print(f'Distance between city 0 and city 4 is : {costs[0, 4]}')
costs

Distance between city 2 and city 5 is : 188
Distance between city 0 and city 4 is : 256


array([[  0, 105,  95, 340, 256, 292],
       [105,   0, 372, 288, 145, 309],
       [ 95, 372,   0, 155, 200, 188],
       [340, 288, 155,   0, 244, 327],
       [256, 145, 200, 244,   0,  98],
       [292, 309, 188, 327,  98,   0]])

In [243]:
# Fitness score of initial population
fitness_score = compute_fitness_score(costs, population)
best_fitness_score = fitness_score.max()
best_ind_idx = np.argwhere(fitness_score == best_fitness_score)[0][0] # Only takes one individual among all
actual_cost = find_cost(costs, population[best_ind_idx])
best_individual = {'individual': population[best_ind_idx], 'actual_cost': actual_cost} 
fitness_score

array([393., 470., 539., 411., 535., 480., 458., 539., 371., 396., 300.,
       547., 822., 542., 371., 565., 323., 175., 500., 355., 535., 271.,
        81., 316., 359., 444., 412., 353.,  21.,   0., 176., 595., 406.,
       686., 177., 616., 169., 154., 368., 264., 281., 602., 272., 495.,
       219., 345.,  40., 355., 297., 449., 348., 484., 407., 227., 582.,
       398., 348., 383., 411., 242., 502., 411., 329., 587., 266., 150.,
       529., 353., 384., 328., 433., 529., 406., 538., 688.,  73., 444.,
       538., 679., 359., 257., 402., 258., 728., 315., 398., 328., 265.,
       337., 297., 211., 529., 535., 322.,  18., 323., 117., 282.,  48.,
       278.])

In [244]:
# Running Genetic Algorithm
print('Start running Genetic Algorithm...')
i=1
while i <= epochs:
    # Generate new population by doing mating then compute their fitness_score
    population = mating(population, fitness_score, crossover_rate, mutation_rate)
    fitness_score = compute_fitness_score(costs, population)

    #####################################################################################################################
    # Keep track of the best individual among all the generations 
    # Here should compaire their actual costs and not their fitness_score since the max value used to compute the fitness score for 
    # each individual varies from generation to generation.
    # Remember in TSP smaller the cost (or distance) --> better the solution
    best_ind_generation_idx = np.argwhere(fitness_score == fitness_score.max())[0][0] # Only takes one individual among all 
    best_ind_gen = population[best_ind_generation_idx]
    actual_cost = find_cost(costs, best_ind_gen)
    if actual_cost < best_individual['actual_cost']:
        best_individual['individual'] = population[best_ind_generation_idx]
        best_individual['actual_cost'] = actual_cost
    #####################################################################################################################
   
    # Printing epoch...
    if i % show_info_after == 0:
        print(f'Epoch {i} done...')
    i += 1 

Start running Genetic Algorithm...
Epoch 1000 done...
Epoch 2000 done...
Epoch 3000 done...
Epoch 4000 done...
Epoch 5000 done...
Epoch 6000 done...
Epoch 7000 done...
Epoch 8000 done...
Epoch 9000 done...
Epoch 10000 done...


In [245]:
# Fitness score last generation
fitness_score

array([346., 471., 254., 375., 290., 325., 597., 320., 195., 418., 452.,
        51., 365., 486., 471., 486., 216., 709., 661., 316., 714., 464.,
       452., 191., 466., 290., 733., 424., 477., 102., 463., 418., 486.,
       304., 610., 693., 661., 804., 494., 520., 550., 463., 375., 520.,
       251., 247., 123., 227., 721., 668.,  48., 202., 102., 216., 416.,
       398., 458.,   0., 524., 436., 363., 386., 411., 392., 597., 221.,
       517., 287., 466., 290., 424., 312., 426., 670., 280., 155., 197.,
       418., 298., 238., 319., 353., 610., 257., 458., 154., 597., 511.,
       282., 665., 665., 587., 334., 554., 197., 517., 346., 377., 347.,
       517.])

In [246]:
# Best individual in the last generation
best_fit_last_gen = fitness_score.max()
idx = np.argwhere(fitness_score == best_fit_last_gen)[0][0] # Only takes one individual among all 
best_ind_last_gen = population[idx]
print(f'Best Individual last generation :{best_ind_last_gen} with fitness score: {best_fit_last_gen}')

Best Individual last generation :[3 2 5 4 1 0] with fitness score: 804.0


In [247]:
# Actual Cost or Distance to be travelled by best individual in the last generation
# For actual_cost smaller is better
find_cost(costs, best_ind_last_gen)

691

In [248]:
# Best Individual among all generations
# For actual_cost smaller is better
best_individual

{'individual': array([3, 2, 0, 1, 4, 5]), 'actual_cost': 598}

<p>
    <i>
        <strong>Note: </strong> 
        Please note that the output values may vary with each execution. This is because genetic algorithms are inherently stochastic in nature and do not guarantee identical results every time they are run.
    </i>   
</p>

<p style="float:right;"><i>Created By Maroyi Bisoka on 24/01/2025</i></p>